In [1]:
import os, pickle, tempfile
import pandas as pd

# Data

In [2]:
extra_set_path = "../../training_data/8.Apos/Extra_set"

In [3]:
with open(f"{extra_set_path}/apos.pkl", "rb") as f:
    apos = pickle.load(f)

len(apos), apos

(9,
 {'8jp0': {'8sgj': ['A']},
  '8uk6': {'AF-A0A1D8PQM9-F1': ['A']},
  '8f4s': {'7l6r': ['A']},
  '8qni': {'8vw5': ['A']},
  '7gqu': {'6yhr': ['A']},
  '7yg5': {'7xlq': ['A']},
  '8aq6': {'5b0u': ['A']},
  '8v81': {'5uak': ['A']},
  '9dnm': {'4jqi': ['A']}})

In [4]:
with open(f"{extra_set_path}/pockets_features.pkl", "rb") as f:
    news = pickle.load(f)

len(news), news

(9,
 {'8sgj':    Pockets                                              Label      FPocket  \
         pdb    pocket nres site_in_pocket pocket_in_site label Pocket Score   
  0     8sgj  pocket16   12        0.00000       0.000000     0      -0.0342   
  1     8sgj  pocket28   14        0.00000       0.000000     0      -0.1447   
  2     8sgj  pocket18   13        0.03125       0.076923     0      -0.0508   
  3     8sgj  pocket11   19        0.00000       0.000000     0       0.0417   
  4     8sgj  pocket25   13        0.00000       0.000000     0      -0.1152   
  5     8sgj  pocket15    9        0.03125       0.111111     0      -0.0332   
  6     8sgj   pocket7    8        0.03125       0.125000     0       0.1316   
  7     8sgj  pocket22   10        0.00000       0.000000     0      -0.0848   
  8     8sgj  pocket13    8        0.00000       0.000000     0       0.0031   
  9     8sgj  pocket23   40        0.00000       0.000000     0      -0.1057   
  10    8sgj  pocket14    9 

In [5]:
with open(f"{extra_set_path}/apos_sites.pkl", "rb") as f:
    news_sites = {k: [{"site": s["site"]} for s in v.values()] for k,v in pickle.load(f).items() if k in news}

len(news_sites), news_sites

(9,
 {'8sgj': [{'site':    label_comp_id label_asym_id label_entity_id label_seq_id pdbx_PDB_ins_code  \
    0            GLU             A               1          132                 ?   
    1            THR             A               1          133                 ?   
    2            VAL             A               1          134                 ?   
    3            SER             A               1          135                 ?   
    4            LEU             A               1          137                 ?   
    5            THR             A               1          138                 ?   
    6            ALA             A               1          141                 ?   
    7            HIS             A               1          200                 ?   
    8            VAL             A               1          203                 ?   
    9            VAL             A               1          206                 ?   
    10           THR             A           

In [6]:
assert all(len(sites) == 1 for sites in news_sites.values()), "Not all apos have a single annotated site"

In [7]:
models = pd.read_pickle("models_lenient_labelling.pkl")

models

{'model5': {'results': {'4jqi': {'pocket16': {'prob': 0.0001226096646860242,
     'pred': 0,
     'label': 0,
     'max_overlap': 0.0,
     'pocket_in_site': 0.0,
     'site_in_pocket': 0.0},
    'pocket18': {'prob': 0.003527346532791853,
     'pred': 0,
     'label': 0,
     'max_overlap': 0.0,
     'pocket_in_site': 0.0,
     'site_in_pocket': 0.0},
    'pocket11': {'prob': 4.818243542104028e-05,
     'pred': 0,
     'label': 0,
     'max_overlap': 0.0,
     'pocket_in_site': 0.0,
     'site_in_pocket': 0.0},
    'pocket15': {'prob': 1.1008408629109567e-09,
     'pred': 0,
     'label': 0,
     'max_overlap': 0.0,
     'pocket_in_site': 0.0,
     'site_in_pocket': 0.0},
    'pocket7': {'prob': 0.0006957116420380771,
     'pred': 0,
     'label': 0,
     'max_overlap': 0.0,
     'pocket_in_site': 0.0,
     'site_in_pocket': 0.0},
    'pocket13': {'prob': 0.09678139537572861,
     'pred': 0,
     'label': 0,
     'max_overlap': 0.0,
     'pocket_in_site': 0.0,
     'site_in_pocket': 0.

In [8]:
colors = {
    "orange": "#D55E00".lower(),
    "green": "#009E73".lower(),
    "blue": "#0072B2".lower()
}

# Labelling

In [9]:
# Percentage of residues of "one" in "other"
get_overlap = lambda one, other: (
    len( one.merge(other) ) / len(one)
)

get_overlaps = lambda pdb, pocketd: {
    name: get_overlap(*one_in_other) 
        for site in news_sites[pdb] 
            for name, one_in_other in (
                ("pocket_in_site", (pocketd["residues"], site["site"])),
                ("site_in_pocket", (site["site"], pocketd["residues"])),
            )
}

def get_label(overlaps, site_in_pocket=None, pocket_in_site=None):
    assert not (site_in_pocket==None and pocket_in_site==None)
    
    if site_in_pocket is None:
        return int( overlaps["pocket_in_site"] >= pocket_in_site )
    if pocket_in_site is None:
        return int( overlaps["site_in_pocket"] >= site_in_pocket )
    return int( overlaps["site_in_pocket"] >= site_in_pocket or overlaps["pocket_in_site"] >= pocket_in_site )

In [10]:
# def label_results(resultsd, site_in_pocket=0.65, pocket_in_site=None, prob_key=None):
#     return pd.DataFrame((
#         {
#             "pdb": pdb,
#             "pocket": pocket,
#             **{"prob": pocketd[prob_key] for prob_key in (prob_key,) if prob_key is not None},
#             "pred": pocketd["pred"],
#             "label": get_label(overlaps, site_in_pocket, pocket_in_site),
#             "max_overlap": max(overlaps.values()),
#             **overlaps,
#         }
#         for pdb, pockets in resultsd.items()
#         for pocket, pocketd in pockets.items()
#         for overlaps in (get_overlaps(pdb, pocketd),)
#     )).sort_values("max_overlap", ascending=False)

def label_results(resultsd, site_in_pocket=0.65, pocket_in_site=None, prob_key=None):
    df = pd.DataFrame((
        {
            "pdb": pdb,
            "pocket": pocket,
            **{"prob": pocketd[prob_key] for prob_key in (prob_key,) if prob_key is not None},
            "pred": pocketd["pred"],
            "label": get_label(overlaps, site_in_pocket, pocket_in_site),
            "max_overlap": max(overlaps.values()),
            **overlaps,
        }
        for pdb, pockets in resultsd.items()
        for pocket, pocketd in pockets.items()
        for overlaps in (get_overlaps(pdb, pocketd),)
    ))
    if prob_key is not None:
        df["pred"] = (
            # Start from a Series where each value/row (sample/pocket) is the total number of pos. labels on its PDB
            df.groupby("pdb")["label"].transform("sum")
            # Then subtract this "total num. of pos. in a PDB" by the rank of each pocket in a PDB, sorted by the probability
            .sub(df.groupby("pdb")["prob"].rank(method="first", ascending=False))
            # If the subtraction is positive or 0 it means that the pocket is in the topX and will be assigned 1
            >= 0
        ).astype(int)
    
    return df.sort_values("max_overlap", ascending=False)

In [11]:
# def label_our_results(resultsd, site_in_pocket=0.65, pocket_in_site=None):
#     return pd.DataFrame((
#         {
#             "pdb": pdb,
#             "pocket": pocket,
#             "prob": pocketd["prob"],
#             "pred": pocketd["pred"],
#             "pred_top1": int( pocketd["prob"] == pdb_maxprob ),
#             "label": get_label(overlaps, site_in_pocket, pocket_in_site),
#             "max_overlap": max(overlaps.values()),
#             **overlaps,
#         }
#         for pdb, pockets in resultsd.items()
#         for pdb_maxprob in (max(pktd["prob"] for pktd in pockets.values()),)
#         for pocket, pocketd in pockets.items()
#         for overlaps in ({k: pocketd[k] for k in ["pocket_in_site", "site_in_pocket"]},)
#     )).sort_values("max_overlap", ascending=False)

def label_our_results(resultsd, site_in_pocket=0.65, pocket_in_site=None):
    df = pd.DataFrame((
        {
            "pdb": pdb,
            "pocket": pocket,
            "prob": pocketd["prob"],
            "label": get_label(overlaps, site_in_pocket, pocket_in_site),
            "max_overlap": max(overlaps.values()),
            **overlaps,
        }
        for pdb, pockets in resultsd.items()
        for pocket, pocketd in pockets.items()
        for overlaps in ({k: pocketd[k] for k in ["pocket_in_site", "site_in_pocket"]},)
    ))
    df["pred"] = (
        # Start from a Series where each value/row (sample/pocket) is the total number of pos. labels on its PDB
        df.groupby("pdb")["label"].transform("sum")
        # Then subtract this "total num. of pos. in a PDB" by the rank of each pocket in a PDB, sorted by the probability
        .sub(df.groupby("pdb")["prob"].rank(method="first", ascending=False))
        # If the subtraction is positive or 0 it means that the pocket is in the topX and will be assigned 1
        >= 0
    ).astype(int)

    return df.sort_values("max_overlap", ascending=False)

In [12]:
models["allositepro"]["results"]

{'7l6r': {'pocket0': {'pred': 1,
   'Volume': 2222.145,
   'SASA': 1065.897,
   'Druggability Score': 0.576,
   'logitProb': 0.844,
   'nmaScore': 0.971,
   'hitScore': 0.869,
   'residues':    auth_asym_id auth_seq_id
   0             A        6972
   1             A        6935
   2             A        6996
   3             A        6947
   4             A        6932
   5             A        6931
   6             A        6999
   7             A        6913
   8             A        6897
   9             A        6824
   10            A        6946
   11            A        6871
   12            A        6896
   13            A        6872
   14            A        6911
   15            A        6844
   16            A        6823
   17            A        6929
   18            A        6971
   19            A        6822
   20            A        6828
   21            A        6841
   22            A        6912
   23            A        6928
   24            A        6937
   25 

In [13]:
for model, modeld in models.items():
    if model != "model5":
        print(model)
        models[model]["labelled"] = label_results(
            modeld["results"], 
            **modeld["labelling"], 
            prob_key=modeld["prob_key"]
        )
    else:
        models[model]["labelled"] = label_our_results(
            modeld["results"], 
            **modeld["labelling"]
        )

allositepro
stingallo
passer_ensemble
passer_automl
passer_rank
deepallo
alloses
mefallosite
allo


# Pocket functions

In [14]:
import sys

sys.path.append("../../training_data")

In [15]:
from utils.utils import Cif, CifFileWriter
from utils.pocket_utils import Pocket

In [16]:
from biotite.structure.io.pdb import PDBFile

def get_pdb_atoms(f):
    atom_array = PDBFile.read(f).get_structure()
    return pd.DataFrame({
            "auth_asym_id": atom_array.chain_id,
            "auth_seq_id": atom_array.res_id,
            "auth_comp_id": atom_array.res_name,
            "auth_atom_id": atom_array.atom_name,
            "type_symbol": atom_array.element,
            "Cartn_x": atom_array.coord[0][:, 0],
            "Cartn_y": atom_array.coord[0][:, 1],
            "Cartn_z": atom_array.coord[0][:, 2],
            "pdbx_PDB_ins_code": (ic or '?' for ic in atom_array.ins_code)
        }, dtype=str)

In [17]:
def get_our_pocket(pdb, pocket, color):
    pocketn = pocket.replace('pocket', '')
    pocket_atoms = (
        Cif(pdb, f"{extra_set_path}/pockets/{pdb}/{pdb}_out/{pdb}_out.cif", name=f"{pdb}_out")
        .atoms
        .query(f"label_comp_id == 'STP' and label_seq_id == '{pocketn}'")
    )
    pocket_atoms["label_entity_id"] = '99'

    return {
        # "number": int(pocketn), 
        "atoms": pocket_atoms, 
        "representation": [{
            'entity_id': '99', "auth_asym_id": pocket_atoms.auth_asym_id.unique().item(), 'auth_residue_number': int(pocketn), 'representation': 'molecular-surface', 'representationColor': colors[color]
        },]
    }#"color": colors[color]}

models["model5"]["pocketf"] = get_our_pocket

In [18]:
def get_allositepro_pocket(pdb, pocket, color):
    resultsf = next(f for f in os.listdir(f"AllositePro/{pdb}") if f.endswith("_download"))
    # pockets are 0-indexed but residue numbers start at 1
    # also pocket0 can be residue ID 2, so all pockets auth_seq_id will be sorted and then the relevant residue id taken with the 0-index pocket number
    pockets_atoms = (
        get_pdb_atoms(f"AllositePro/{pdb}/{resultsf}/{resultsf.replace('_download', '')}.pdb")
        .query(f"auth_comp_id == 'STP'")
    )
    pocketn = sorted(pockets_atoms.auth_seq_id.unique())[ int(pocket.replace('pocket', '')) ]
    pocket_atoms = pockets_atoms.query(f"auth_seq_id == '{pocketn}'")
    
    pocket_atoms["label_entity_id"] = '99'
    return {
        # "number": int(pocketn), 
        "atoms": pocket_atoms, 
        "representation": [{
            'entity_id': '99', "auth_asym_id": pocket_atoms.auth_asym_id.unique().item(), 'auth_residue_number': int(pocketn), 'representation': 'molecular-surface', 'representationColor': colors[color]
        },]
        # "color": colors[color]
    }

models["allositepro"]["pocketf"] = get_allositepro_pocket

In [19]:
from functools import partial

In [20]:
def get_passer_pocket(pdb, pocket, color, model):
    pocketn = pocket.replace('pocket', '')
    pocket_atoms = (
        get_pdb_atoms(f"PASSer/{model}/{pdb}/{pdb}_out.pdb")
        .query(f"auth_comp_id == 'STP' and auth_seq_id == '{pocketn}'")
    )
    
    pocket_atoms["label_entity_id"] = '99'
    return {
        # "number": int(pocketn), 
        "atoms": pocket_atoms, 
        "representation": [{
            'entity_id': '99', "auth_asym_id": pocket_atoms.auth_asym_id.unique().item(), 'auth_residue_number': int(pocketn), 'representation': 'molecular-surface', 'representationColor': colors[color]
        },]
        # "color": colors[color]
    }

models["passer_ensemble"]["pocketf"] = partial(get_passer_pocket, model="ensemble")
models["passer_automl"]["pocketf"] = partial(get_passer_pocket, model="automl")
models["passer_rank"]["pocketf"] = partial(get_passer_pocket, model="rank")

In [21]:
# def get_alloses_pocket(pdb, pocket, color):
#     pocketn = pocket.replace('pocket', '')
#     pocket_atoms = (
#         get_pdb_atoms(f"AllosES/{pdb.upper()}/{pdb.upper()}_out/{pdb.upper()}_out.pdb")
#         .query(f"auth_comp_id == 'STP' and auth_seq_id == '{pocketn}'")
#     )
    
#     pocket_atoms["label_entity_id"] = '99'
#     return {
#         # "number": int(pocketn), 
#         "atoms": pocket_atoms, 
#         "representation": [{
#             'entity_id': '99', "auth_asym_id": pocket_atoms.auth_asym_id.unique().item(), 'auth_residue_number': int(pocketn), 'representation': 'molecular-surface', 'representationColor': colors[color]
#         },]
#         # "color": colors[color]
#     }

# models["alloses"]["pocketf"] = get_alloses_pocket

In [22]:
def get_allo_pocket(pdb, pocket, color):
    if "AF" not in pdb:
        resultsf = next(f for f in os.listdir(f"ALLO/{pdb}") if f.startswith(f"{pdb}pdb") and f.endswith("_desc.txt")).replace("_desc.txt", "")
    else:
        afpdb = pdb.lower().replace("-", "")
        resultsf = next(f for f in os.listdir(f"ALLO/{pdb}") if f.startswith(f"{afpdb}pdb") and f.endswith("_desc.txt")).replace("_desc.txt", "")
    pocket_atoms = (
        get_pdb_atoms(f"ALLO/{pdb}/pockets/{resultsf}_{pocket}_res.pdb")
    )
    
    pocket_atoms["label_entity_id"] = '99'
    pocket_atoms["auth_asym_id"] = 'ZZZ'
    return {
        # "number": int(pocketn), 
        "atoms": pocket_atoms.query("auth_atom_id not in ['CA', 'C', 'O', 'N', 'CB']"), 
        "representation": [
            {
                'auth_asym_id': "ZZZ", 
                "auth_residue_number": int(res),
                'representation': 'molecular-surface',
                'representationColor': colors[color]
            }
            for res in pocket_atoms["auth_seq_id"].unique()
        ]
        # "color": colors[color]
    }

models["allo"]["pocketf"] = get_allo_pocket

In [23]:
def get_mefallosite_pocket(pdb, pocket, color):
    pocketn = pocket.replace('pocket', '')
    pocket_atoms = (
        get_pdb_atoms(f"MEF-AlloSite/MEF-AlloSite/data/test/{pdb.upper()}_cleaned_out/{pdb.upper()}_cleaned_out.pdb")
        .query(f"auth_comp_id == 'STP' and auth_seq_id == '{pocketn}'")
    )
    
    pocket_atoms["label_entity_id"] = '99'
    return {
        # "number": int(pocketn), 
        "atoms": pocket_atoms, 
        "representation": [{
            'entity_id': '99', "auth_asym_id": pocket_atoms.auth_asym_id.unique().item(), 'auth_residue_number': int(pocketn), 'representation': 'molecular-surface', 'representationColor': colors[color]
        },]
        # "color": colors[color]
    }

models["mefallosite"]["pocketf"] = get_mefallosite_pocket

# View functions

In [24]:
from ipymolstar import PDBeMolstar

In [25]:
# ass_fields_list = ["_pdbx_struct_assembly", "_pdbx_struct_assembly_gen", "_pdbx_struct_oper_list"]

def view_pockets(pdb, pockets:list):
    cif = Cif(pdb, f"{extra_set_path}/origcifs/{pdb}_updated.cif.gz")

    # minimal_elements = lambda element="label_asym_id": site["site"][element].unique().tolist() + site["mod"][element].unique().tolist()

    site = news_sites[pdb][0]
    # atoms = cif.atoms.query(f"label_asym_id in {minimal_elements('label_asym_id')}")

    # Fake entity data
    entities = pd.concat((
        pd.DataFrame(cif.cif.data["_entity"], dtype=str),#.query(f"id in {minimal_elements('label_entity_id')}"),
        pd.DataFrame([{"id": "99", "type": "branched", "pdbx_description": "pockets"}]) # Fake the pockets as carbohydrates to manage their representation
    )).fillna(".")

    

    columns = list( set.intersection( *map(set, (pocket_atoms["atoms"].columns for pocket_atoms in pockets)) ) )
    atoms = pd.concat((
        cif.atoms[columns],
        *(pocket_atoms["atoms"][columns] for pocket_atoms in pockets)
    ))

    with tempfile.NamedTemporaryFile("w+", suffix=".cif") as f:
        writer = CifFileWriter(f.name)
        writer.write({cif.entry_id.upper(): {
            "_entity": entities.to_dict(orient="list"),
            "_atom_site": atoms.to_dict(orient="list"),
            # **{k: cif.cif.data[k] for k in ass_fields_list}
        }})
        combined = Cif(pdb, filename=f.name)
        combined.cif.data # to cache it while 'f' exists
        
    v = PDBeMolstar(
        custom_data = {
                'data': combined.cif.text,
                'format': 'cif',
                'binary': False,
            },
        sequence_panel = True,
        assembly_id='', # str(ass_ids[pdb]),
        
        hide_polymer = True,
        hide_heteroatoms = True,
        hide_carbs = True,
        hide_water = True,
        hide_non_standard = True,
        
        color_data = {
            "data": [
                # Protein
                *(
                    {"auth_asym_id": asym_id, 'representation': 'cartoon', 'representationColor': '#AEAEAE'} #, 'focus': True
                    for asym_id in site["site"].auth_asym_id.unique().tolist()
                ),
                # Site
                *(
                    {'auth_asym_id': r["auth_asym_id"], 'auth_residue_number': int(r["auth_seq_id"]) if r["auth_seq_id"] != "." else "", 'representationColor': colors["green"], 'focus': True}
                    for i, r in site["site"].iterrows()
                    if pdb != "8sgj" or (pdb == "8sgj" and int(r["auth_seq_id"]) < 793) ############################################
                ),
                # # Modulator
                # *(
                #     {"entity_id": entity_id, 'representation': "spacefill", }
                #     for entity_id in entities.query("type == 'non-polymer'").id.unique()
                #     if entity_id in site["mod"].label_entity_id.unique()
                # ),         
                # Other ligands
                *(
                    {"entity_id": entity_id, 'representation': "ball-and-stick", }
                    for entity_id in entities.query("type == 'non-polymer'").id.unique()
                ),               
                # Pockets
                *(rep for pocket in pockets for rep in pocket["representation"]),
                # *(
                #     {'entity_id': '99', "auth_asym_id": pocket["atoms"].auth_asym_id.unique().item(), 'auth_residue_number': pocket["number"], 'representation': 'molecular-surface', 'representationColor': pocket["color"]}
                #     for pocket in pockets
                # ),
            ],
            "nonSelectedColor": None,
            "keepColors": False,
            "keepRepresentations": False,
        },
    )
    
    return v

In [26]:
models["model5"]["labelled"]

,pdb,pocket,prob,label,max_overlap,pocket_in_site,site_in_pocket,pred
46,5uak,pocket1,0.986737,1,0.962963,0.139785,0.962963,1
195,7xlq,pocket36,0.007708,1,0.950000,0.950000,0.655172,0
252,AF-A0A1D8PQM9-F1,pocket1,0.763694,1,0.904762,0.475000,0.904762,1
113,6yhr,pocket12,0.000328,0,0.750000,0.750000,0.310345,0
230,8vw5,pocket1,0.081857,1,0.695652,0.551724,0.695652,1
...,...,...,...,...,...,...,...,...
93,5uak,pocket63,0.000105,0,0.000000,0.000000,0.000000,0
78,5uak,pocket45,0.000309,0,0.000000,0.000000,0.000000,0
79,5uak,pocket12,0.131247,0,0.000000,0.000000,0.000000,0
80,5uak,pocket50,0.000002,0,0.000000,0.000000,0.000000,0


In [27]:
def view_top(pdb, model, top=None):
    pocketf = models[model]["pocketf"]
    prob_key = models[model]["prob_key"]
    results = models[model]["results"][pdb]
    labelled = models[model]["labelled"].query(f"pdb == '{pdb}'").sort_values("prob", ascending=False)
    pos_pockets = labelled[labelled["label"] == 1]
    
    if top is None:
        top = labelled["label"].sum() or 1
    top_pockets = tuple(pocket for i, pocket in tuple(labelled.iterrows())[:top])

    for pocket in top_pockets:
        print(pocket["pocket"], {k: v for k, v in results[pocket["pocket"]].items() if k != "residues"}, "label:", pocket["label"])

    return view_pockets(
        pdb,
        tuple(
            [
                pocketf(pdb, pocket["pocket"], "green" if pocket["label"] == 1 else "blue")
                for pocket in top_pockets
            ] + [
                pocketf(pdb, pocket["pocket"], "orange")
                for i, pocket in pos_pockets.iterrows()
                if pocket["pocket"] not in (p["pocket"] for p in top_pockets)
            ]
        )
    )

# Viz

- Components settings: Ignore light
- Examine ligands and make selection+component or hide all
- Duplicate spacefill (modulator), add stick-and-ball and hide spacefill. Carbon color uniform: ~30,30,30
- Adjust molecular surface Probe radius and Opacity (0.4)

In [28]:
apos

{'8jp0': {'8sgj': ['A']},
 '8uk6': {'AF-A0A1D8PQM9-F1': ['A']},
 '8f4s': {'7l6r': ['A']},
 '8qni': {'8vw5': ['A']},
 '7gqu': {'6yhr': ['A']},
 '7yg5': {'7xlq': ['A']},
 '8aq6': {'5b0u': ['A']},
 '8v81': {'5uak': ['A']},
 '9dnm': {'4jqi': ['A']}}

In [29]:
modell = ["model5", "allo", "mefallosite"]#"allositepro",

In [75]:
i = 0

In [78]:
curr_model = modell[i]; print(curr_model)
i += 1
v = view_top('4jqi', curr_model)#, top=20)
v

mefallosite
pocket1 {'prob': 0.6268527656793594, 'pred': 1} label: 0


PDBeMolstar(bg_color='#F7F7F7', color_data={'data': [{'auth_asym_id': 'A', 'representation': 'cartoon', 'repre…